In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import joblib, pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, get_scorer_names
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso
from xgboost import XGBRegressor

# 데이터 전처리

## 면적, 생산량만 추출

In [3]:
df = pd.read_csv("data/진혁_데이터프레임.csv")
# df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496 entries, 0 to 1495
Data columns (total 65 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1496 non-null   object 
 1   Unnamed: 1              1496 non-null   object 
 2   Unnamed: 2              1496 non-null   int64  
 3   평균기온(°C)_1월             1496 non-null   float64
 4   최고기온(°C)_1월             1496 non-null   float64
 5   최저기온(°C)_1월             1496 non-null   float64
 6   월합강수량(00~24h만)(mm)_1월   1496 non-null   float64
 7   합계 일사량(MJ/m2)_1월        1496 non-null   float64
 8   평균기온(°C)_2월             1496 non-null   float64
 9   최고기온(°C)_2월             1496 non-null   float64
 10  최저기온(°C)_2월             1496 non-null   float64
 11  월합강수량(00~24h만)(mm)_2월   1496 non-null   float64
 12  합계 일사량(MJ/m2)_2월        1496 non-null   float64
 13  평균기온(°C)_3월             1496 non-null   float64
 14  최고기온(°C)_3월             1496 non-null   

,Unnamed: 0,Unnamed: 1,Unnamed: 2,평균기온(°C)_1월,최고기온(°C)_1월,최저기온(°C)_1월,월합강수량(00~24h만)(mm)_1월,합계 일사량(MJ/m2)_1월,평균기온(°C)_2월,최고기온(°C)_2월,최저기온(°C)_2월,월합강수량(00~24h만)(mm)_2월,합계 일사량(MJ/m2)_2월,평균기온(°C)_3월,최고기온(°C)_3월,최저기온(°C)_3월,월합강수량(00~24h만)(mm)_3월,합계 일사량(MJ/m2)_3월,평균기온(°C)_4월,최고기온(°C)_4월,최저기온(°C)_4월,월합강수량(00~24h만)(mm)_4월,합계 일사량(MJ/m2)_4월,평균기온(°C)_5월,최고기온(°C)_5월,최저기온(°C)_5월,월합강수량(00~24h만)(mm)_5월,합계 일사량(MJ/m2)_5월,평균기온(°C)_6월,최고기온(°C)_6월,최저기온(°C)_6월,월합강수량(00~24h만)(mm)_6월,합계 일사량(MJ/m2)_6월,평균기온(°C)_7월,최고기온(°C)_7월,최저기온(°C)_7월,월합강수량(00~24h만)(mm)_7월,합계 일사량(MJ/m2)_7월,평균기온(°C)_8월,최고기온(°C)_8월,최저기온(°C)_8월,월합강수량(00~24h만)(mm)_8월,합계 일사량(MJ/m2)_8월,평균기온(°C)_9월,최고기온(°C)_9월,최저기온(°C)_9월,월합강수량(00~24h만)(mm)_9월,합계 일사량(MJ/m2)_9월,평균기온(°C)_10월,최고기온(°C)_10월,최저기온(°C)_10월,월합강수량(00~24h만)(mm)_10월,합계 일사량(MJ/m2)_10월,평균기온(°C)_11월,최고기온(°C)_11월,최저기온(°C)_11월,월합강수량(00~24h만)(mm)_11월,합계 일사량(MJ/m2)_11월,평균기온(°C)_12월,최고기온(°C)_12월,최저기온(°C)_12월,월합강수량(00~24h만)(mm)_12월,합계 일사량(MJ/m2)_12월,면적 (ha),생산량 (톤)
0,강원도,spring,2000,-2.90,8.5,-16.0,48.1,152.990,-3.3,7.7,-15.80,5.70,329.07,4.40,19.30,-8.6,14.10,448.65,10.40,23.7,-3.1,37.10,486.355,15.90,31.6,3.90,75.20,520.280,21.3,33.60,9.60,116.9,547.195,24.90,34.30,16.7,153.4,512.23,24.1,33.7,17.40,348.0,412.735,18.5,31.1,8.5,256.90,347.645,12.1,26.1,-1.6,16.90,311.070,4.1,20.30,-8.4,30.2,231.67,-1.40,12.50,-14.9,16.9,234.16,2147.0,87619.0
1,강원도,spring,2001,-2.65,10.2,-15.0,52.2,202.280,-2.3,8.3,-15.05,1.55,317.51,5.25,19.75,-8.6,2.55,455.28,10.85,23.7,-2.2,23.55,530.950,17.05,31.4,5.55,67.55,562.770,22.7,33.35,12.65,113.6,554.200,25.75,34.05,18.8,244.0,542.24,25.0,34.0,18.25,417.4,453.020,19.0,31.2,9.2,160.85,394.070,12.6,26.8,-1.7,18.55,359.460,4.7,20.75,-7.9,25.0,262.15,-0.55,13.25,-12.9,24.1,248.43,1983.0,83405.0
2,강원도,spring,2002,1.40,13.2,-9.6,21.9,173.530,1.2,13.5,-9.70,0.00,360.91,7.30,19.70,-7.3,33.10,519.21,12.70,24.8,-1.2,50.00,542.480,17.70,34.1,5.70,65.00,586.320,21.9,32.80,12.20,133.5,471.260,25.80,34.10,18.4,384.0,529.20,25.7,34.6,19.70,415.0,463.430,20.4,32.1,9.1,316.10,389.160,15.2,26.8,3.2,38.00,374.260,7.9,21.60,-4.2,56.0,301.48,2.00,16.10,-8.3,2.5,287.57,1329.0,56908.0
3,강원도,spring,2003,-0.30,10.7,-12.6,29.2,167.555,0.2,10.4,-12.90,0.00,398.05,6.60,21.30,-6.0,28.50,523.00,11.90,25.9,-1.4,49.00,540.765,17.50,34.5,5.60,52.60,586.195,21.9,34.90,9.50,111.5,504.755,24.90,35.10,16.5,179.6,491.39,24.3,34.0,17.60,244.6,414.440,18.6,31.1,8.0,250.00,325.020,14.0,25.6,0.6,22.50,311.485,6.4,20.70,-6.9,51.5,229.38,0.40,14.80,-11.2,4.9,235.80,1735.0,82760.0
4,강원도,spring,2004,1.00,15.3,-8.8,33.7,176.490,0.6,12.4,-7.60,15.40,366.47,6.90,20.30,-4.9,19.90,481.75,12.60,25.4,-0.5,16.30,537.210,17.80,33.9,5.80,47.70,540.490,22.5,32.70,13.60,197.7,359.160,26.50,34.30,20.3,321.4,305.84,26.3,33.9,18.70,538.3,258.010,20.4,30.6,11.0,228.70,212.690,15.9,28.1,4.5,31.70,235.700,8.4,22.10,-3.0,53.8,295.02,3.20,16.70,-6.6,6.4,294.42,2133.0,100486.0


In [5]:
df = df.drop(columns=df.columns[:-2])
df = df.dropna(how="any")

## scaling 및 train, test 분리

In [7]:
feature = df["면적 (ha)"]
target = df["생산량 (톤)"]

In [9]:
mmsc_t = MinMaxScaler()
mmsc_f = MinMaxScaler()

mmsc_t.fit(feature.values.reshape(-1,1))
mmsc_f.fit(target.values.reshape(-1,1))

X = mmsc_t.transform(feature.values.reshape(-1,1))
y = mmsc_f.transform(target.values.reshape(-1,1))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

# 모델 학습

## GridSearchCV를 통한 최적의 파라미터 탐색

In [19]:
lasso = Lasso()
ridge = Ridge()
xgb = XGBRegressor()

alpha = np.logspace(-4,2,7)   # lasso, ridge 해당
max_iter = [1,3,5,10,20,50,100,150,200]   # lasso, ridge 해당
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']   # ridge 해당, 계산에 사용할 알고리즘
eta = np.logspace(-3,2,6)   # xgboost 해당
n_estimators = [15,20,50,100,200,500]   # randomforesteregressor(사용할 tree의 개수), gradientboost, adaboost, xgboost 해당
max_depth = [2,3,5,7,10,12]  # randomforesteregressor(tree의 깊이), gradientboost, xgboost 해당
gamma = [0, 0.05, 0.1, 0.2, 0.5]    # xgboost 해당
max_leaf_nodes = [2,3,5,7]     # randomforest, xgboost 해당
min_child_weight = [1,2,3,5]   # xgboost 해당

params_lasso = {"alpha" : alpha, 
                "max_iter" : max_iter}

params_ridge = {"alpha" : alpha, 
                "max_iter" : max_iter, 
                "solver" : solver}

params_xgb = {"learning_rate" : eta,
                "n_estimators" : n_estimators, 
                "max_depth" : max_depth, 
                "gamma" : gamma,
                "min_child_weight" : min_child_weight}

In [11]:
kfold = KFold(5, shuffle=True, random_state=6)

## lasso 모델 생성

In [13]:
grid_lasso = GridSearchCV(estimator=lasso, param_grid=params_lasso, n_jobs=-1, cv=kfold, scoring="neg_mean_squared_error")
grid_lasso.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=6, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'max_iter': [1, 3, 5, 10, 20, 50, 100, 150, 200]},
             scoring='neg_mean_squared_error')

In [14]:
print("best score :", grid_lasso.best_score_)
print("best parameter :", grid_lasso.best_params_)

best score : -0.004609469940299442
best parameter : {'alpha': 0.0001, 'max_iter': 1}


In [15]:
em = grid_lasso.best_estimator_
y_pred = em.predict(X_test)
print("r2 score :", r2_score(y_test, y_pred))

r2 score : 0.8216685017595488


In [16]:
y_pred_train = em.predict(X_train)
print("r2 score :", r2_score(y_train, y_pred_train))

r2 score : 0.8142548802096018


## ridge 모델 생성

In [21]:
grid_ridge = GridSearchCV(estimator=ridge, param_grid=params_ridge, n_jobs=-1, cv=kfold, scoring="neg_mean_squared_error")
grid_ridge.fit(X_train, y_train)

c:\Users\jhahn\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
315 fits failed out of a total of 2520.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
315 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jhahn\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jhahn\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 1130, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "c:\Users\jhahn\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 806, in fit
    raise ValueError(
ValueError: 'lbfgs' sol

GridSearchCV(cv=KFold(n_splits=5, random_state=6, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'max_iter': [1, 3, 5, 10, 20, 50, 100, 150, 200],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag', 'saga', 'lbfgs']},
             scoring='neg_mean_squared_error')

In [23]:
print("best score :", grid_ridge.best_score_)
print("best parameter :", grid_ridge.best_params_)

best score : -0.004571889909901773
best parameter : {'alpha': 0.0001, 'max_iter': 100, 'solver': 'sag'}


In [24]:
em = grid_ridge.best_estimator_
y_pred = em.predict(X_test)
print("r2 score :", r2_score(y_test, y_pred))

r2 score : 0.8222894487576194


In [25]:
y_pred_train = em.predict(X_train)
print("r2 score :", r2_score(y_train, y_pred_train))

r2 score : 0.8142764466618546


## xgbregressor 모델 생성

In [26]:
grid_xgb = GridSearchCV(estimator=xgb, param_grid=params_xgb, n_jobs=-1, cv=kfold, scoring="neg_mean_squared_error")
grid_xgb.fit(X_train, y_train)

c:\Users\jhahn\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [-0.19500846 -0.19311464 -0.18214588 ...         nan         nan
         nan]
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=6, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    inter...
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'gamma': [0, 0.05, 0.1, 0.2, 0.5],
                         'learning_rate': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'max_depth': [2, 3, 5, 7, 10, 12],
                         'min_child_weight': [1, 2, 3, 5],
                         'n_estimators': [15, 20, 50, 100, 200, 500]},
             scoring='neg_mean_squared_error')

In [27]:
print("best score :", grid_xgb.best_score_)
print("best parameter :", grid_xgb.best_params_)

best score : -0.00324759163506044
best parameter : {'gamma': 0, 'learning_rate': 0.01, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 500}


In [28]:
em = grid_xgb.best_estimator_
y_pred = em.predict(X_test)
print("r2 score :", r2_score(y_test, y_pred))

r2 score : 0.8923914825883453


In [29]:
y_pred_train = em.predict(X_train)
print("r2 score :", r2_score(y_train, y_pred_train))

r2 score : 0.9217935397151706


# 모델, 스케일러 저장